<a href="https://colab.research.google.com/github/vaibhav-prasad707/DAA_707/blob/main/transferLearning%2BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import (
    ResNet50, resnet50,
    VGG16, vgg16,
    MobileNetV2, mobilenet_v2,
    InceptionV3, inception_v3,
    DenseNet121, densenet
)
from tensorflow.keras.utils import to_categorical

### Model Configurations

In [2]:
MODEL_CONFIG = {
    'ResNet50': (ResNet50, resnet50.preprocess_input, (224, 224)),
    'VGG16': (VGG16, vgg16.preprocess_input, (224, 224)),
    'MobileNetV2': (MobileNetV2, mobilenet_v2.preprocess_input, (224, 224)),
    'DenseNet121': (DenseNet121, densenet.preprocess_input, (224, 224)),
    'InceptionV3': (InceptionV3, inception_v3.preprocess_input, (299, 299)),
}

### Mock data (For now)

In [3]:
def get_mock_data(num_samples, img_size, num_classes):
    """
    Generates mock image data and labels.
    """
    print(f"Generating {num_samples} mock samples of size {img_size}...")
    # Generate random pixel data
    x = np.random.rand(num_samples, img_size[0], img_size[1], 3).astype(np.float32)
    # Generate random integer labels
    y_int = np.random.randint(0, num_classes, size=num_samples)
    # One-hot encode labels
    y_one_hot = to_categorical(y_int, num_classes)
    return x, y_one_hot

### LSTM Model

In [4]:
def create_hybrid_cnn_lstm_model(model_name, num_classes):
    """
    Builds a modular hybrid CNN-LSTM model.

    Args:
        model_name (str): The name of the model from MODEL_CONFIG.
        num_classes (int): The number of output classes.

    Returns:
        tf.keras.models.Model: The compiled Keras model.
    """

    # 1. Get model configuration
    if model_name not in MODEL_CONFIG:
        raise ValueError(f"Model {model_name} not found in MODEL_CONFIG.")

    ModelClass, preprocess_fn, img_size = MODEL_CONFIG[model_name]
    input_shape = (img_size[0], img_size[1], 3)

    # 2. Load pre-trained base model
    base_model = ModelClass(
        weights='imagenet',
        include_top=False,     # Exclude the final classification layer
        input_shape=input_shape
    )
    # Freeze the weights of the pre-trained layers
    base_model.trainable = False

    # 3. Build the full hybrid model
    inputs = layers.Input(shape=input_shape, name='input_image')

    # 4. Apply model-specific preprocessing
    # This Lambda layer bakes the preprocessing into the model
    x = layers.Lambda(preprocess_fn, name='preprocessing')(inputs)

    # 5. Pass through the frozen base model
    x = base_model(x, training=False)

    # 6. Reshape CNN output for LSTM
    # The CNN base outputs a 4D feature map: (batch, height, width, channels)
    # We need to reshape it into a 3D sequence for the LSTM: (batch, timesteps, features)
    # We'll treat the spatial dimensions (height * width) as the "timesteps"

    # Get the output shape of the base model
    # e.g., (None, 7, 7, 2048) for ResNet50
    cnn_output_shape = base_model.output_shape

    # Calculate timesteps and features
    # timesteps = height * width (e.g., 7 * 7 = 49)
    # features = channels (e.g., 2048)
    timesteps = cnn_output_shape[1] * cnn_output_shape[2]
    features = cnn_output_shape[3]

    # Reshape: (None, 7, 7, 2048) -> (None, 49, 2048)
    x = layers.Reshape((timesteps, features), name='reshape_for_lstm')(x)

    # 7. Add LSTM layer
    # The LSTM will find temporal/spatial patterns in the features
    x = layers.LSTM(64, name='lstm_layer')(x)

    # 8. Add final classification layer
    outputs = layers.Dense(num_classes, activation='softmax', name='classifier')(x)

    # 9. Create and compile the final model
    model = models.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.AUC(name='auc'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )

    return model

### Experiment

In [5]:
def run_experiment(model_name, num_classes, epochs, batch_size):
    """
    Runs a full experiment for a single model.
    """
    print("-" * 80)
    print(f"Running experiment for: {model_name}")
    print("-" * 80)

    # 1. Get model-specific input size
    img_size = MODEL_CONFIG[model_name][2]

    # 2. Load data (using mock data here)
    # Replace this with your actual data loading logic
    # Make sure your real images are resized to `img_size` *before* a batch
    x_train, y_train = get_mock_data(100, img_size, num_classes)
    x_val, y_val = get_mock_data(20, img_size, num_classes)
    x_test, y_test = get_mock_data(20, img_size, num_classes)

    # 3. Create the model
    model = create_hybrid_cnn_lstm_model(model_name, num_classes)

    if model_name == 'ResNet50':
        print("\nModel Summary (ResNet50 example):")
        model.summary()

    # 4. Train the model
    print(f"\nTraining {model_name}...")
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    # 5. Evaluate the model
    print(f"\nEvaluating {model_name}...")
    results = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)

    # Format results into a dictionary
    metrics = model.metrics_names
    results_dict = dict(zip(metrics, results))

    print(f"Evaluation Results for {model_name}:")
    for key, value in results_dict.items():
        print(f"  {key}: {value:.4f}")
    print("-" * 80)

    return results_dict

### Main Comparative Analysis


In [6]:
if __name__ == "__main__":

    # --- Configuration ---
    # Define which models you want to compare
    MODELS_TO_COMPARE = ['MobileNetV2', 'ResNet50', 'DenseNet121']
    NUM_CLASSES = 3  # e.g., (Nevus, Melanoma, Keratosis)
    EPOCHS = 2       # Keep low for a quick test. Increase for real training.
    BATCH_SIZE = 16

    all_results = {}

    for model_name in MODELS_TO_COMPARE:
        results = run_experiment(model_name, NUM_CLASSES, EPOCHS, BATCH_SIZE)
        all_results[model_name] = results

    # Print final comparative summary
    print("\n" + "=" * 80)
    print("           FINAL COMPARATIVE ANALYSIS")
    print("=" * 80)

    # Convert results to a pandas DataFrame for nice printing
    results_df = pd.DataFrame.from_dict(all_results, orient='index')
    results_df = results_df.round(4)

    print(results_df)

--------------------------------------------------------------------------------
Running experiment for: MobileNetV2
--------------------------------------------------------------------------------
Generating 100 mock samples of size (224, 224)...
Generating 20 mock samples of size (224, 224)...
Generating 20 mock samples of size (224, 224)...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

Training MobileNetV2...
Epoch 1/2
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.3485 - auc: 0.5764 - loss: 1.1458 - precision: 0.3626 - recall: 0.3014 - val_accuracy: 0.2000 - val_auc: 0.4469 - val_loss: 1.1392 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/2
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 687ms/step - accuracy: 0.4147 - auc: 0.5119 - loss: 1.1338 - precision: 0.2272 - recall: 0.0970 - val_accuracy: 0.2000 - val_auc: 0.3888 - val_loss: 1.2712 - val_precision: 0.2000 - val_recall: 0.2000

Evaluating MobileNetV2...
Evaluation Results for MobileNetV2:
  loss: 1.2512
  compile_metrics:

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ preprocessing (Lambda)          │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_for_lstm (Reshape)      │ (None, 49, 2048)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_layer (LSTM)               │ (None, 64)             │       540,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classifier (Dense)              │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,128,835 (92.04 MB)

 Trainable params: 541,123 (2.06 MB)

 Non-trainable params: 23,587,712 (89.98 MB)


Training ResNet50...
Epoch 1/2
7/7 ━━━━━━━━━━━━━━━━━━━━ 34s 4s/step - accuracy: 0.3342 - auc: 0.5242 - loss: 1.1709 - precision: 0.3865 - recall: 0.1940 - val_accuracy: 0.1500 - val_auc: 0.3169 - val_loss: 1.3416 - val_precision: 0.1500 - val_recall: 0.1500
Epoch 2/2
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.4522 - auc: 0.5981 - loss: 1.0660 - precision: 0.4805 - recall: 0.2777 - val_accuracy: 0.5000 - val_auc: 0.5706 - val_loss: 1.0910 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00

Evaluating ResNet50...
Evaluation Results for ResNet50:
  loss: 1.1093
  compile_metrics: 0.2500
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Running experiment for: DenseNet121
--------------------------------------------------------------------------------
Generating 100 mock samples of size (224, 224)...
Generating 20 mock samples of size (224, 224)...
Generating 20 mock s